In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import Row, SQLContext, SparkSession
from pyspark import SparkConf
# Import data types
from pyspark.sql.types import *

In [ ]:
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"

In [ ]:
conf = SparkConf()
conf.setAppName("pcap_analyzer")
conf.setMaster("k8s://https://192.168.50.123:6443")
conf.set("spark.executor.instances","2")
conf.set("spark.kubernetes.container.image", "smileocean/spark-py:spark_3.0.0_preview2_python")
conf.set("spark.kubernetes.namespace", "spark-employee")
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName","spark-employee")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate()

In [ ]:
#multipleline
pcapPath = "/home/antslab/data_hdd4t/中華電信/pcap/snort.2020-01-10_dir/snort.log/outbound_*.json"
pcap_df_multiline = spark.read.option("multiline", "true").json(pcapPath)

In [ ]:
# 重新整理schema
# layer_df = pcap_df_multiline.select("_source.layers.`ip.src`")
# src_ip_df = layer_df.select("`ip.src`")

In [ ]:
src_ip_df = pcap_df_multiline.select("_source.layers.`ip.src`")
src_ip_df.createOrReplaceTempView("src_ip")

In [ ]:
maxflowquery = (spark.sql("SELECT `ip.src`, COUNT(*) FROM src_ip GROUP BY `ip.src` ORDER BY COUNT(*) DESC LIMIT 10"))

In [ ]:
# 直接拿maxflowquery來畫圖
result = maxflowquery.collect()

In [ ]:
import pickle as pk
with open("cht_0110_top10.pk", 'wb') as file:
    pk.dump(result, file)

In [ ]:
import pickle as pk
with open("/home/antslab/spark_data/intermediate/pcap/cht_0110_top10.pk", 'rb') as file:
    result = pk.load(file)